<a href="https://colab.research.google.com/github/nh0127/SEF_DB/blob/main/SEF_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

식품의약품안전처_의약품 제품 허가정보_의약품 제품 허가 상세 정보

In [ ]:
# (상단 코드 및 CipherSuiteAdapter 클래스는 이전과 동일)
import requests
import json
import math
import time
import os
import ssl
from urllib.parse import unquote
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from google.colab import userdata

class CipherSuiteAdapter(HTTPAdapter):
    def __init__(self, *args, **kwargs):
        CIPHERS = ('ECDH+AESGCM:DH+AESGCM:ECDH+AES256:DH+AES256:ECDH+AES128:DH+AES:ECDH+HIGH:'
                   'DH+HIGH:RSA+AESGCM:RSA+AES:RSA+HIGH:!aNULL:!eNULL:!MD5:!3DES')
        self.ssl_context = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
        self.ssl_context.set_ciphers(CIPHERS)
        super().__init__(*args, **kwargs)

    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = requests.packages.urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize, block=block, ssl_context=self.ssl_context)

def download_all_drug_data_in_one_go(service_key, save_filename="drug_permission_data_final.json"):
    """
    일일 트래픽(10,000회)이 충분하므로, 하루 만에 모든 데이터를 수집하는 최종 함수.
    """
    api_url_base = "https://apis.data.go.kr"
    api_path = "/1471000/DrugPrdtPrmsnInfoService06/getDrugPrdtPrmsnDtlInq05"
    full_api_url = api_url_base + api_path
    decoded_service_key = unquote(service_key)

    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
    adapter = CipherSuiteAdapter(max_retries=retries)
    session.mount(api_url_base, adapter)

    all_items = []
    rows_per_page = 100

    # 1. 1페이지를 요청하여 전체 데이터 수를 정확히 파악
    print("데이터 수집을 시작합니다...")
    try:
        params = {'serviceKey': decoded_service_key, 'pageNo': '1', 'numOfRows': '1', 'type': 'json'}
        response = session.get(full_api_url, params=params, timeout=30)
        response.raise_for_status()
        data = response.json()
        if data['header']['resultCode'] != '00':
            print(f"API 에러: {data['header']['resultMsg']}")
            return False
        total_count = int(data['body']['totalCount'])
        total_pages = math.ceil(total_count / rows_per_page)
        print(f"총 데이터: {total_count}개, 전체 페이지: {total_pages}개를 수집합니다.")
        print("-" * 50)
    except requests.exceptions.RequestException as e:
        print(f"초기 정보 파악 실패: {e}"); session.close(); return False

    # 2. 1페이지부터 마지막 페이지까지 순차적으로 수집
    for page in range(1, total_pages + 1):
        try:
            params = {'serviceKey': decoded_service_key, 'pageNo': str(page), 'numOfRows': str(rows_per_page), 'type': 'json'}
            print(f"페이지 {page}/{total_pages} 수집 중...")
            response = session.get(full_api_url, params=params, timeout=30)
            response.raise_for_status()
            data = response.json()

            if data['header']['resultCode'] == '00' and 'items' in data['body'] and data['body']['items']:
                items_data = data['body']['items']
                if not isinstance(items_data, list): items_data = [items_data]
                all_items.extend(items_data)
            else:
                 print(f"  -> 페이지 {page}에서 데이터 없음: {data['header'].get('resultMsg', '데이터 없음')}")

            # 서버 부하를 줄이기 위한 최소한의 딜레이
            time.sleep(0.2)

            # 100 페이지마다 긴 휴식을 주어 안정성 확보
            if page % 100 == 0 and page != total_pages:
                print(f"  -> {page} 페이지 도달. 서버 부하 방지를 위해 10초간 휴식합니다...")
                time.sleep(10)

        except (requests.exceptions.RequestException, json.JSONDecodeError) as e:
            print(f"  -> !!! 페이지 {page} 수집 실패: {e}. 이 페이지를 건너뜁니다.")
            continue

    # 3. 최종 저장
    session.close()
    try:
        print("-" * 50)
        print(f"총 {len(all_items)}개의 데이터를 성공적으로 수집했습니다.")
        print(f"'{save_filename}' 파일로 최종 저장합니다...")
        with open(save_filename, 'w', encoding='utf-8') as f:
            json.dump(all_items, f, ensure_ascii=False, indent=2)
        print(f"저장 완료! 모든 데이터 수집이 완료되었습니다!")
        return True
    except Exception as e:
        print(f"최종 파일 저장 중 에러 발생: {e}"); return False

if __name__ == "__main__":
    MY_SERVICE_KEY = userdata.get('MY_SERVICE_KEY')
    download_all_drug_data_in_one_go(service_key=MY_SERVICE_KEY)

데이터 수집을 시작합니다...
총 데이터: 44248개, 전체 페이지: 443개를 수집합니다.
--------------------------------------------------
페이지 1/443 수집 중...
페이지 2/443 수집 중...
페이지 3/443 수집 중...
페이지 4/443 수집 중...
페이지 5/443 수집 중...
페이지 6/443 수집 중...
페이지 7/443 수집 중...
페이지 8/443 수집 중...
페이지 9/443 수집 중...
페이지 10/443 수집 중...
페이지 11/443 수집 중...
페이지 12/443 수집 중...
페이지 13/443 수집 중...
페이지 14/443 수집 중...
페이지 15/443 수집 중...
페이지 16/443 수집 중...
페이지 17/443 수집 중...
페이지 18/443 수집 중...
페이지 19/443 수집 중...
페이지 20/443 수집 중...
페이지 21/443 수집 중...
페이지 22/443 수집 중...
페이지 23/443 수집 중...
페이지 24/443 수집 중...
페이지 25/443 수집 중...
페이지 26/443 수집 중...
페이지 27/443 수집 중...
페이지 28/443 수집 중...
페이지 29/443 수집 중...
페이지 30/443 수집 중...
페이지 31/443 수집 중...
페이지 32/443 수집 중...
페이지 33/443 수집 중...
페이지 34/443 수집 중...
페이지 35/443 수집 중...
페이지 36/443 수집 중...
페이지 37/443 수집 중...
페이지 38/443 수집 중...
페이지 39/443 수집 중...
페이지 40/443 수집 중...
페이지 41/443 수집 중...
페이지 42/443 수집 중...
페이지 43/443 수집 중...
페이지 44/443 수집 중...
페이지 45/443 수집 중...
페이지 46/443 수집 중...
페이지 47/443 수집 중...
페이지 48/443 수

페이지 294/443 수집 중...
페이지 295/443 수집 중...
페이지 296/443 수집 중...
페이지 297/443 수집 중...
페이지 298/443 수집 중...
페이지 299/443 수집 중...
페이지 300/443 수집 중...
  -> 300 페이지 도달. 서버 부하 방지를 위해 10초간 휴식합니다...
페이지 301/443 수집 중...
페이지 302/443 수집 중...
페이지 303/443 수집 중...
페이지 304/443 수집 중...
페이지 305/443 수집 중...
페이지 306/443 수집 중...
페이지 307/443 수집 중...
페이지 308/443 수집 중...
페이지 309/443 수집 중...
페이지 310/443 수집 중...
페이지 311/443 수집 중...
페이지 312/443 수집 중...
페이지 313/443 수집 중...
페이지 314/443 수집 중...
페이지 315/443 수집 중...
페이지 316/443 수집 중...
페이지 317/443 수집 중...
페이지 318/443 수집 중...
페이지 319/443 수집 중...
페이지 320/443 수집 중...
페이지 321/443 수집 중...
페이지 322/443 수집 중...
페이지 323/443 수집 중...
페이지 324/443 수집 중...
페이지 325/443 수집 중...
페이지 326/443 수집 중...
페이지 327/443 수집 중...
페이지 328/443 수집 중...
페이지 329/443 수집 중...
페이지 330/443 수집 중...
페이지 331/443 수집 중...
페이지 332/443 수집 중...
페이지 333/443 수집 중...
페이지 334/443 수집 중...
페이지 335/443 수집 중...
페이지 336/443 수집 중...
페이지 337/443 수집 중...
페이지 338/443 수집 중...
페이지 339/443 수집 중...
페이지 340/443 수집 중...
페이지 341/443 수집 중

In [ ]:
import pandas as pd
import json
import re

print("새로운 데이터 전처리를 시작합니다...")

# 1단계: 파일 로드
try:
    df = pd.read_json('drug_permission_data_final.json', encoding='utf-8')
    print(f"파일 로드 성공! 원본 데이터 형태: {df.shape}")
except FileNotFoundError:
    print("drug_permission_data_final.json 파일을 찾을 수 없습니다. 파일이 현재 폴더에 있는지 확인하세요.")
    exit() # 파일이 없으면 종료

# 2단계: 기본 정제
# ------------------------------------------------------------------------------
# 2.1. 컬럼 이름을 소문자로 통일 (e.g., ITEM_NAME -> item_name)
df.columns = [col.lower() for col in df.columns]
print("\n컬럼 이름을 소문자로 변경했습니다.")

# 2.2. 고유 ID(item_seq) 기준으로 중복 데이터 제거
before_dedup = len(df)
df.drop_duplicates(subset=['item_seq'], keep='first', inplace=True)
after_dedup = len(df)
print(f"중복 제거: {before_dedup}개 -> {after_dedup}개 ({before_dedup - after_dedup}개 제거)")

# 2.3. 주요 텍스트 컬럼의 결측값(NaN)을 빈 문자열('')로 처리
text_cols = ['ee_doc_data', 'ud_doc_data', 'nb_doc_data', 'ingr_name']
df[text_cols] = df[text_cols].fillna('')
print("결측값 처리를 완료했습니다.")

# 3단계: 정보 구조화 및 표준화 (Feature Engineering)
# ------------------------------------------------------------------------------

# 3.1. 제품명(item_name)에서 괄호와 성분명 제거
def remove_brackets(name):
    """제품명에서 괄호와 그 안의 내용만 제거하는 함수"""
    cleaned_name = name
    # 중첩 괄호를 처리하기 위해 5번 반복 (대부분의 경우 충분)
    for _ in range(5):
        if '(' not in cleaned_name: break
        cleaned_name = re.sub(r'\s*\([^)]*\)', '', cleaned_name)
    return cleaned_name.strip()

df['item_name_clean'] = df['item_name'].apply(remove_brackets)
print("\n제품명(item_name) 정제를 완료했습니다.")

# 3.2. 성분명(ingr_name)을 파싱하여 리스트로 만들기
def parse_ingredients(text):
    """쉼표(,)나 세미콜론(;)으로 구분된 성분명을 리스트로 변환하는 함수"""
    if not text:
        return []
    # 구분자를 기준으로 분리하고 각 성분의 앞뒤 공백 제거
    ingredients = [ing.strip() for ing in re.split(r'[,;]', text)]
    # 빈 문자열은 리스트에서 제외
    return [ing for ing in ingredients if ing]

df['ingredients_list'] = df['ingr_name'].apply(parse_ingredients)
print("성분명(ingr_name) 파싱을 완료했습니다.")

# 3.3. 주의사항(nb_doc_data)에서 상호작용 약물 목록 추출
def parse_interactions_from_notes(text):
    """주의사항 텍스트에서 상호작용 약물 목록을 추출하는 함수"""
    if '상호작용' not in text or '병용' not in text:
        return []

    # 다양한 목록 형태(1., 1), •, -)를 처리하는 정규표현식
    pattern = r'(?:[①-⑨\d][\.\)]|•|-)\s*([가-힣a-zA-Z0-9\s/]+?)(?=[,;\n①-⑨\d•-]|$)'
    drugs = re.findall(pattern, text)

    cleaned_drugs = []
    for drug in drugs:
        clean_drug = drug.strip().split(',')[0].strip()
        clean_drug = re.sub(r'\s+등$', '', clean_drug)
        if clean_drug:
            cleaned_drugs.append(clean_drug)

    return cleaned_drugs

df['interactions_list'] = df['nb_doc_data'].apply(parse_interactions_from_notes)
print("주의사항(nb_doc_data)에서 상호작용 정보 추출을 완료했습니다.")


# 4단계: 최종 저장
# ------------------------------------------------------------------------------
# 챗봇에 필요한 최종 컬럼들 선택
final_v2_df = df[[
    'item_seq',
    'item_name',
    'item_name_clean', # 정제된 제품명
    'entp_name',
    'etc_otc_code', # 전문/일반의약품 코드
    'ingr_name', # 원본 성분명 텍스트
    'ingredients_list', # 파싱된 성분명 리스트
    'ee_doc_data', # 효능효과
    'ud_doc_data', # 용법용량
    'nb_doc_data', # 주의사항 원본 텍스트
    'interactions_list' # 추출된 상호작용 리스트
]]

# 효율적인 Parquet 포맷으로 저장
try:
    save_filename = 'drug_data_processed_v2.parquet'
    final_v2_df.to_parquet(save_filename, index=False)
    print(f"\n전처리 완료! 최종 데이터를 '{save_filename}' 파일로 저장했습니다.")
except ImportError:
    # pyarrow 라이브러리가 없는 경우 CSV로 저장
    save_filename = 'drug_data_processed_v2.csv'
    final_v2_df.to_csv(save_filename, index=False, encoding='utf-8-sig')
    print("\n[경고] pyarrow 라이브러리가 없어 CSV로 저장합니다.")
    print(f"전처리 완료! 최종 데이터를 '{save_filename}' 파일로 저장했습니다.")

# 결과 확인
print("\n--- 전처리 결과 샘플 (상호작용 정보가 있는 데이터 위주) ---")
result_check = final_v2_df[final_v2_df['interactions_list'].apply(len) > 0]
if not result_check.empty:
    print(result_check[['item_name_clean', 'ingredients_list', 'interactions_list']].head())
else:
    print("상호작용 정보를 포함한 데이터를 찾지 못했습니다. 원본 데이터를 확인해보세요.")

새로운 데이터 전처리를 시작합니다...
파일 로드 성공! 원본 데이터 형태: (44148, 42)

컬럼 이름을 소문자로 변경했습니다.
중복 제거: 44148개 -> 44144개 (4개 제거)
결측값 처리를 완료했습니다.

제품명(item_name) 정제를 완료했습니다.
성분명(ingr_name) 파싱을 완료했습니다.
주의사항(nb_doc_data)에서 상호작용 정보 추출을 완료했습니다.

전처리 완료! 최종 데이터를 'drug_data_processed_v2.parquet' 파일로 저장했습니다.

--- 전처리 결과 샘플 (상호작용 정보가 있는 데이터 위주) ---
          item_name_clean                 ingredients_list  \
9          제일에페드린염산염주사액4%                  [[M040534]주사용수]   
11              안나카주사액20%                  [[M040534]주사용수]   
12              안나카주사액10%                  [[M040534]주사용수]   
14  대원염산에페드린주사액[수출명:에린주사]                  [[M040534]주사용수]   
15             대원아미노필린주사액  [[M040534]주사용수|[M223366]에틸렌디아민]   

                                    interactions_list  
9   [MAO억제제를 투여중이거나 투여를 중지한 지, 갑상샘기능항진증, 고량투여 시 불안...  
11                                 [MAO억제제와 병용투여시 빈맥]  
12                                 [MAO억제제와 병용투여시 빈맥]  
14  [MAO억제제를 투여중이거나 투여를 중지한 지, 갑상샘기능항진증, 고량투여 시 불안...  
15  [앰플주사제는 용기절단시 유리파편이 혼입

식품의약품안전처_의약품 제품 허가정보_의약품 제품 허가 목록

In [ ]:
# 이전에 드렸던 최종 완성 코드와 동일합니다.
# 이 코드를 다시 실행해주세요.
import requests
import json
import math
import time
import os
import ssl
from urllib.parse import unquote
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from google.colab import userdata

class CipherSuiteAdapter(HTTPAdapter):
    def __init__(self, *args, **kwargs):
        CIPHERS = ('ECDH+AESGCM:DH+AESGCM:ECDH+AES256:DH+AES256:ECDH+AES128:DH+AES:ECDH+HIGH:'
                   'DH+HIGH:RSA+AESGCM:RSA+AES:RSA+HIGH:!aNULL:!eNULL:!MD5:!3DES')
        self.ssl_context = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
        self.ssl_context.set_ciphers(CIPHERS)
        super().__init__(*args, **kwargs)

    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = requests.packages.urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize, block=block, ssl_context=self.ssl_context)

def download_drug_permit_list_data(service_key, save_filename="drug_permit_list_data.json"):
    api_url_base = "https://apis.data.go.kr"
    api_path = "/1471000/DrugPrdtPrmsnInfoService06/getDrugPrdtPrmsnInq06"
    full_api_url = api_url_base + api_path

    decoded_service_key = unquote(service_key)

    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
    adapter = CipherSuiteAdapter(max_retries=retries)
    session.mount(api_url_base, adapter)

    all_items = []
    rows_per_page = 100

    print("새로운 데이터('의약품 제품 허가 목록') 수집을 시작합니다...")
    try:
        params = {'serviceKey': decoded_service_key, 'pageNo': '1', 'numOfRows': '1', 'type': 'json'}
        response = session.get(full_api_url, params=params, timeout=30)
        response.raise_for_status()
        data = response.json()

        if data['header']['resultCode'] != '00':
            print(f"API 에러: {data['header']['resultMsg']}")
            return False

        total_count = int(data['body']['totalCount'])
        total_pages = math.ceil(total_count / rows_per_page)
        print(f"총 데이터: {total_count}개, 전체 페이지: {total_pages}개를 수집합니다.")
        print("-" * 50)
    except requests.exceptions.RequestException as e:
        print(f"초기 정보 파악 실패: {e}"); session.close(); return False

    for page in range(1, total_pages + 1):
        try:
            params = {'serviceKey': decoded_service_key, 'pageNo': str(page), 'numOfRows': str(rows_per_page), 'type': 'json'}
            print(f"페이지 {page}/{total_pages} 수집 중...")
            response = session.get(full_api_url, params=params, timeout=30)
            response.raise_for_status()
            data = response.json()

            if (data['header']['resultCode'] == '00' and 'items' in data['body']
                and data['body']['items']): # 'item' 키가 없을 수도 있으므로 items 까지만 확인

                items_data = data['body']['items'] # items 에 바로 리스트가 올 수도 있음
                if 'item' in items_data: # item 키가 있으면 한 단계 더 들어감
                    items_data = items_data['item']

                if isinstance(items_data, list):
                    all_items.extend(items_data)
                elif isinstance(items_data, dict):
                    all_items.append(items_data)
            else:
                 print(f"  -> 페이지 {page}에서 데이터 없음: {data['header'].get('resultMsg', 'items 키 없음')}")

            time.sleep(0.2)

            if page % 100 == 0 and page != total_pages:
                print(f"  -> {page} 페이지 도달. 서버 부하 방지를 위해 10초간 휴식합니다...")
                time.sleep(10)

        except (requests.exceptions.RequestException, json.JSONDecodeError) as e:
            print(f"  -> !!! 페이지 {page} 수집 실패: {e}. 이 페이지를 건너뜁니다.")
            continue

    session.close()
    try:
        print("-" * 50)
        print(f"총 {len(all_items)}개의 데이터를 성공적으로 수집했습니다.")
        print(f"'{save_filename}' 파일로 최종 저장합니다...")

        with open(save_filename, 'w', encoding='utf-8') as f:
            json.dump(all_items, f, ensure_ascii=False, indent=2)

        print(f"저장 완료! 모든 데이터 수집이 완료되었습니다!")
        return True
    except Exception as e:
        print(f"최종 파일 저장 중 에러 발생: {e}"); return False

# --- 스크립트 실행 부분 ---
if __name__ == "__main__":
    MY_SERVICE_KEY = userdata.get('MY_SERVICE_KEY')
    download_all_drug_data_in_one_go(service_key=MY_SERVICE_KEY)

데이터 수집을 시작합니다...
총 데이터: 44248개, 전체 페이지: 443개를 수집합니다.
--------------------------------------------------
페이지 1/443 수집 중...
페이지 2/443 수집 중...
페이지 3/443 수집 중...
페이지 4/443 수집 중...
페이지 5/443 수집 중...
페이지 6/443 수집 중...
페이지 7/443 수집 중...
페이지 8/443 수집 중...
페이지 9/443 수집 중...
페이지 10/443 수집 중...
페이지 11/443 수집 중...
페이지 12/443 수집 중...
페이지 13/443 수집 중...
페이지 14/443 수집 중...
페이지 15/443 수집 중...
페이지 16/443 수집 중...
페이지 17/443 수집 중...
페이지 18/443 수집 중...
페이지 19/443 수집 중...
페이지 20/443 수집 중...
페이지 21/443 수집 중...
페이지 22/443 수집 중...
페이지 23/443 수집 중...
페이지 24/443 수집 중...
페이지 25/443 수집 중...
페이지 26/443 수집 중...
페이지 27/443 수집 중...
페이지 28/443 수집 중...
페이지 29/443 수집 중...
페이지 30/443 수집 중...
페이지 31/443 수집 중...
페이지 32/443 수집 중...
페이지 33/443 수집 중...
페이지 34/443 수집 중...
페이지 35/443 수집 중...
페이지 36/443 수집 중...
페이지 37/443 수집 중...
페이지 38/443 수집 중...
페이지 39/443 수집 중...
페이지 40/443 수집 중...
페이지 41/443 수집 중...
페이지 42/443 수집 중...
페이지 43/443 수집 중...
페이지 44/443 수집 중...
페이지 45/443 수집 중...
페이지 46/443 수집 중...
페이지 47/443 수집 중...
페이지 48/443 수

페이지 213/443 수집 중...
페이지 214/443 수집 중...
페이지 215/443 수집 중...
페이지 216/443 수집 중...
페이지 217/443 수집 중...
페이지 218/443 수집 중...
페이지 219/443 수집 중...
페이지 220/443 수집 중...
페이지 221/443 수집 중...
페이지 222/443 수집 중...


페이지 223/443 수집 중...
페이지 224/443 수집 중...
페이지 225/443 수집 중...
페이지 226/443 수집 중...
페이지 227/443 수집 중...
페이지 228/443 수집 중...
페이지 229/443 수집 중...
페이지 230/443 수집 중...
페이지 231/443 수집 중...
페이지 232/443 수집 중...
페이지 233/443 수집 중...
페이지 234/443 수집 중...
페이지 235/443 수집 중...
페이지 236/443 수집 중...
페이지 237/443 수집 중...
페이지 238/443 수집 중...
페이지 239/443 수집 중...
페이지 240/443 수집 중...
페이지 241/443 수집 중...
페이지 242/443 수집 중...
페이지 243/443 수집 중...
페이지 244/443 수집 중...
페이지 245/443 수집 중...
페이지 246/443 수집 중...
페이지 247/443 수집 중...
페이지 248/443 수집 중...
페이지 249/443 수집 중...
페이지 250/443 수집 중...
페이지 251/443 수집 중...
페이지 252/443 수집 중...
페이지 253/443 수집 중...
페이지 254/443 수집 중...
페이지 255/443 수집 중...
페이지 256/443 수집 중...
페이지 257/443 수집 중...
페이지 258/443 수집 중...
페이지 259/443 수집 중...
페이지 260/443 수집 중...
페이지 261/443 수집 중...
페이지 262/443 수집 중...
페이지 263/443 수집 중...
페이지 264/443 수집 중...
페이지 265/443 수집 중...
페이지 266/443 수집 중...
페이지 267/443 수집 중...
페이지 268/443 수집 중...
페이지 269/443 수집 중...
페이지 270/443 수집 중...
페이지 271/443 수집 중...
페이지 272/443 수집 중...


페이지 294/443 수집 중...
페이지 295/443 수집 중...
페이지 296/443 수집 중...
페이지 297/443 수집 중...
페이지 298/443 수집 중...
페이지 299/443 수집 중...
페이지 300/443 수집 중...
  -> 300 페이지 도달. 서버 부하 방지를 위해 10초간 휴식합니다...
페이지 301/443 수집 중...
페이지 302/443 수집 중...
페이지 303/443 수집 중...
페이지 304/443 수집 중...
페이지 305/443 수집 중...
페이지 306/443 수집 중...
페이지 307/443 수집 중...
페이지 308/443 수집 중...
페이지 309/443 수집 중...
페이지 310/443 수집 중...
페이지 311/443 수집 중...
페이지 312/443 수집 중...
페이지 313/443 수집 중...
페이지 314/443 수집 중...
페이지 315/443 수집 중...
페이지 316/443 수집 중...
페이지 317/443 수집 중...
페이지 318/443 수집 중...
페이지 319/443 수집 중...
페이지 320/443 수집 중...
페이지 321/443 수집 중...
페이지 322/443 수집 중...
페이지 323/443 수집 중...
페이지 324/443 수집 중...
페이지 325/443 수집 중...
페이지 326/443 수집 중...
페이지 327/443 수집 중...
페이지 328/443 수집 중...
페이지 329/443 수집 중...
페이지 330/443 수집 중...
페이지 331/443 수집 중...
페이지 332/443 수집 중...
페이지 333/443 수집 중...
페이지 334/443 수집 중...
페이지 335/443 수집 중...
페이지 336/443 수집 중...
페이지 337/443 수집 중...
페이지 338/443 수집 중...
페이지 339/443 수집 중...
페이지 340/443 수집 중...
페이지 341/443 수집 중

페이지 388/443 수집 중...
페이지 389/443 수집 중...
페이지 390/443 수집 중...
페이지 391/443 수집 중...
페이지 392/443 수집 중...
페이지 393/443 수집 중...
페이지 394/443 수집 중...
페이지 395/443 수집 중...
페이지 396/443 수집 중...
페이지 397/443 수집 중...
페이지 398/443 수집 중...
페이지 399/443 수집 중...
페이지 400/443 수집 중...
  -> 400 페이지 도달. 서버 부하 방지를 위해 10초간 휴식합니다...
페이지 401/443 수집 중...
페이지 402/443 수집 중...
페이지 403/443 수집 중...
페이지 404/443 수집 중...
페이지 405/443 수집 중...
페이지 406/443 수집 중...
페이지 407/443 수집 중...
페이지 408/443 수집 중...
페이지 409/443 수집 중...
페이지 410/443 수집 중...
페이지 411/443 수집 중...
페이지 412/443 수집 중...
페이지 413/443 수집 중...
페이지 414/443 수집 중...
페이지 415/443 수집 중...
페이지 416/443 수집 중...
페이지 417/443 수집 중...
페이지 418/443 수집 중...
페이지 419/443 수집 중...
페이지 420/443 수집 중...
페이지 421/443 수집 중...
페이지 422/443 수집 중...
페이지 423/443 수집 중...
페이지 424/443 수집 중...
페이지 425/443 수집 중...
페이지 426/443 수집 중...
페이지 427/443 수집 중...
페이지 428/443 수집 중...
페이지 429/443 수집 중...
페이지 430/443 수집 중...
페이지 431/443 수집 중...
페이지 432/443 수집 중...
페이지 433/443 수집 중...
페이지 434/443 수집 중...
페이지 435/443 수집 중

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import json
import re

print("새로운 데이터('의약품 제품 허가 목록') 전처리를 시작합니다...")

# 1단계: 파일 로드
try:
    df = pd.read_json('drug_permit_list_data.json', encoding='utf-8')
    print(f"파일 로드 성공! 원본 데이터 형태: {df.shape}")
except FileNotFoundError:
    print("drug_permit_list_data.json 파일을 찾을 수 없습니다. 파일이 현재 폴더에 있는지 확인하세요.")
    exit()

# 2단계: 기본 정제
# ------------------------------------------------------------------------------
# 2.1. 컬럼 이름을 스네이크 케이스로 통일 (일관성 유지)
df.columns = [col.lower() for col in df.columns]
print("\n컬럼 이름을 소문자로 변경했습니다.")

# 2.2. 고유 ID(item_seq) 기준으로 중복 데이터 제거
before_dedup = len(df)
df.drop_duplicates(subset=['item_seq'], keep='first', inplace=True)
after_dedup = len(df)
print(f"중복 제거: {before_dedup}개 -> {after_dedup}개 ({before_dedup - after_dedup}개 제거)")

# 2.3. 주요 텍스트 컬럼의 결측값(NaN)을 빈 문자열('')로 처리
text_cols = ['item_eng_name', 'entp_eng_name', 'item_ingr_name', 'cancel_name']
df[text_cols] = df[text_cols].fillna('')
print("결측값 처리를 완료했습니다.")

# 3단계: 정보 구조화 및 표준화 (Feature Engineering)
# ------------------------------------------------------------------------------

# 3.1. 제품명(item_name)에서 괄호와 성분명 제거
def remove_brackets(name):
    """제품명에서 괄호와 그 안의 내용만 제거하는 함수"""
    if not isinstance(name, str): return "" # 문자열이 아닌 경우 대비
    cleaned_name = name
    for _ in range(5):
        if '(' not in cleaned_name: break
        cleaned_name = re.sub(r'\s*\([^)]*\)', '', cleaned_name)
    return cleaned_name.strip()

df['item_name_clean'] = df['item_name'].apply(remove_brackets)
print("\n제품명(item_name) 정제를 완료했습니다.")

# 3.2. 주성분명(item_ingr_name)을 파싱하여 리스트로 만들기
#    이 데이터의 주성분명은 '성분1,성분2, ...' 형태로 되어 있을 가능성이 높습니다.
def parse_main_ingredients(text):
    """쉼표(,)로 구분된 주성분명을 리스트로 변환하는 함수"""
    if not text or not isinstance(text, str):
        return []
    # 구분자를 기준으로 분리하고 각 성분의 앞뒤 공백 제거
    ingredients = [ing.strip() for ing in text.split(',')]
    # 빈 문자열은 리스트에서 제외
    return [ing for ing in ingredients if ing]

df['main_ingredients_list'] = df['item_ingr_name'].apply(parse_main_ingredients)
print("주성분명(item_ingr_name) 파싱을 완료했습니다.")


# 4단계: 최종 저장
# ------------------------------------------------------------------------------
# 이 데이터에서 유용하게 사용할 컬럼들 선택
final_df = df[[
    'item_seq',
    'item_name',
    'item_name_clean', # 정제된 제품명
    'entp_name',
    'spclty_pblc', # 전문/일반의약품 구분
    'prduct_type', # 제품 타입 (예: 완제의약품)
    'item_ingr_name', # 원본 주성분 텍스트
    'main_ingredients_list', # 파싱된 주성분 리스트
    'big_prdt_img_url', # 제품 이미지 URL
    'edi_code' # EDI 코드 (보험코드)
]]

# 효율적인 Parquet 포맷으로 저장
try:
    save_filename = 'drug_data_processed_v3.parquet'
    final_df.to_parquet(save_filename, index=False)
    print(f"\n전처리 완료! 최종 데이터를 '{save_filename}' 파일로 저장했습니다.")
except ImportError:
    save_filename = 'drug_data_processed_v3.csv'
    final_df.to_csv(save_filename, index=False, encoding='utf-8-sig')
    print(f"\n[경고] pyarrow 라이브러리가 없어 CSV로 저장합니다.")
    print(f"전처리 완료! 최종 데이터를 '{save_filename}' 파일로 저장했습니다.")

# 결과 확인
print("\n--- 전처리 결과 샘플 ---")
print(final_df[['item_name_clean', 'spclty_pblc', 'main_ingredients_list']].head())

새로운 데이터('의약품 제품 허가 목록') 전처리를 시작합니다...
파일 로드 성공! 원본 데이터 형태: (44248, 21)

컬럼 이름을 소문자로 변경했습니다.
중복 제거: 44248개 -> 44244개 (4개 제거)
결측값 처리를 완료했습니다.

제품명(item_name) 정제를 완료했습니다.
주성분명(item_ingr_name) 파싱을 완료했습니다.

전처리 완료! 최종 데이터를 'drug_data_processed_v3.parquet' 파일로 저장했습니다.

--- 전처리 결과 샘플 ---
  item_name_clean spclty_pblc      main_ingredients_list
0    중외5%포도당생리식염액       전문의약품  [Glucose/Sodium Chloride]
1      중외5%포도당주사액       전문의약품                  [Glucose]
2     중외20%포도당주사액       전문의약품                  [Glucose]
3     중외50%포도당주사액       전문의약품                  [Glucose]
4        대한포도당주사액       전문의약품                  [Glucose]


식품의약품안전처_의약품 제품 허가정보_의약품 제품 주성분 상세정보

In [ ]:
import requests
import json
import math
import time
import ssl
from urllib.parse import unquote
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from google.colab import userdata

# SSL/TLS 암호화 스위트 관련 문제를 해결하기 위한 어댑터 클래스
# data.go.kr 서버와 통신 시 간혹 발생하는 SSL 오류를 우회합니다.
class CipherSuiteAdapter(HTTPAdapter):
    def __init__(self, *args, **kwargs):
        # 특정 암호화 스위트 목록을 지정합니다.
        CIPHERS = ('ECDH+AESGCM:DH+AESGCM:ECDH+AES256:DH+AES256:ECDH+AES128:DH+AES:ECDH+HIGH:'
                   'DH+HIGH:RSA+AESGCM:RSA+AES:RSA+HIGH:!aNULL:!eNULL:!MD5:!3DES')
        self.ssl_context = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
        self.ssl_context.set_ciphers(CIPHERS)
        super().__init__(*args, **kwargs)

    def init_poolmanager(self, connections, maxsize, block=False, **pool_kwargs):
        # 커스텀 SSL 컨텍스트를 PoolManager에 적용합니다.
        pool_kwargs['ssl_context'] = self.ssl_context
        super().init_poolmanager(connections, maxsize, block, **pool_kwargs)


def download_all_main_component_data(service_key, save_filename="drug_main_component_data.json"):
    """
    '의약품 제품 주성분 상세정보' API의 모든 데이터를 안정적으로 수집하여 JSON 파일로 저장합니다.
    - 자동 재시도, 서버 부하 감소, SSL 오류 처리 기능이 포함되어 있습니다.
    """
    # --- 1. API 요청 설정 ---
    api_url_base = "https://apis.data.go.kr"
    api_path = "/1471000/DrugPrdtPrmsnInfoService06/getDrugPrdtMcpnDtlInq06" # 목표 API 경로
    full_api_url = api_url_base + api_path

    # 공공데이터포털에서 발급된 키는 URL 인코딩된 상태이므로, 디코딩하여 사용합니다.
    decoded_service_key = unquote(service_key)

    # --- 2. 안정적인 통신을 위한 세션 설정 ---
    session = requests.Session()

    # 서버 오류(5xx) 발생 시, 최대 5번까지 1초 간격으로 자동 재시도를 설정합니다.
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])

    # 위에서 정의한 CipherSuiteAdapter와 재시도 설정을 세션에 탑재합니다.
    adapter = CipherSuiteAdapter(max_retries=retries)
    session.mount(api_url_base, adapter)

    all_items = []
    rows_per_page = 100

    # --- 3. 전체 데이터 수 파악 (첫 페이지 요청) ---
    print("데이터 수집을 시작합니다...")
    try:
        params = {'serviceKey': decoded_service_key, 'pageNo': '1', 'numOfRows': '1', 'type': 'json'}
        response = session.get(full_api_url, params=params, timeout=30)
        response.raise_for_status() # HTTP 에러 발생 시 예외 발생
        data = response.json()

        if data['header']['resultCode'] != '00':
            print(f"API 에러: {data['header']['resultMsg']}")
            return False

        total_count = int(data['body']['totalCount'])
        total_pages = math.ceil(total_count / rows_per_page)

        print(f"총 데이터: {total_count}개, 전체 페이지: {total_pages}개를 수집합니다. (페이지당 {rows_per_page}개)")
        print("-" * 60)

    except requests.exceptions.RequestException as e:
        print(f"초기 정보 파악 실패: {e}")
        session.close()
        return False

    # --- 4. 모든 페이지 순회하며 데이터 수집 ---
    for page in range(1, total_pages + 1):
        try:
            params = {
                'serviceKey': decoded_service_key,
                'pageNo': str(page),
                'numOfRows': str(rows_per_page),
                'type': 'json'
            }
            print(f"페이지 {page}/{total_pages} 수집 중...")

            response = session.get(full_api_url, params=params, timeout=30)
            response.raise_for_status()
            data = response.json()

            # 응답이 정상이면서 'items' 데이터가 실제로 존재할 경우
            if data['header']['resultCode'] == '00' and 'items' in data['body'] and data['body']['items']:
                items_data = data['body']['items']
                all_items.extend(items_data)
            else:
                # 데이터가 없는 경우 (API의 일시적 문제 등)
                print(f"  -> 페이지 {page}에서 데이터 없음: {data['header'].get('resultMsg', '응답에 items 없음')}")

            # 서버 부하를 줄이기 위한 최소한의 딜레이 (0.1초)
            time.sleep(0.1)

            # 100 페이지마다 긴 휴식을 주어 안정성 확보
            if page % 100 == 0 and page != total_pages:
                print(f"  -> {page} 페이지 도달. 서버 부하 방지를 위해 5초간 휴식합니다...")
                time.sleep(5)

        except (requests.exceptions.RequestException, json.JSONDecodeError) as e:
            print(f"  -> !!! 페이지 {page} 수집 실패: {e}. 이 페이지를 건너뜁니다.")
            continue # 실패 시 다음 페이지로 넘어감

    # --- 5. 수집 완료 후 파일로 저장 ---
    session.close()
    try:
        print("-" * 60)
        print(f"총 {len(all_items)}개의 데이터를 성공적으로 수집했습니다.")
        print(f"'{save_filename}' 파일로 최종 저장합니다...")

        # UTF-8 인코딩 및 가독성을 위한 옵션으로 JSON 파일 저장
        with open(save_filename, 'w', encoding='utf-8') as f:
            json.dump(all_items, f, ensure_ascii=False, indent=2)

        print("저장 완료! 모든 데이터 수집이 성공적으로 끝났습니다. 🎉")
        return True

    except Exception as e:
        print(f"최종 파일 저장 중 에러 발생: {e}")
        return False

# --- 스크립트 실행 부분 ---
if __name__ == "__main__":
    MY_SERVICE_KEY = userdata.get('MY_SERVICE_KEY')
    download_all_drug_data_in_one_go(service_key=MY_SERVICE_KEY)

In [ ]:
import pandas as pd
import json

print("새로운 데이터('의약품 주성분 상세정보') 전처리를 시작합니다...")

# 1단계: 파일 로드 (동일)
try:
    df = pd.read_json('drug_main_component_data.json', encoding='utf-8')
    print(f"파일 로드 성공! 원본 데이터 형태: {df.shape}")
except FileNotFoundError:
    print("drug_main_component_data.json 파일을 찾을 수 없습니다.")
    exit()

# 2단계: 기본 정제 (이전과 동일)
# ------------------------------------------------------------------------------
df.columns = [col.lower() for col in df.columns]
print("\n컬럼 이름을 소문자로 변경했습니다.")

df.drop_duplicates(subset=['item_seq', 'mtral_code'], keep='first', inplace=True)
print(f"중복 제거 후 데이터 수: {len(df)}개")

df['mtral_nm'] = df['mtral_nm'].fillna('정보 없음')
df['prduct'] = df['prduct'].fillna('제품명 없음') # 제품명 결측값 처리 추가
df['qnt'] = df['qnt'].astype(str)
print("'qnt' 컬럼을 정보 손실 없이 문자열 타입으로 변환했습니다.")

# --- [핵심 수정 부분] ---
# 3단계: 정보 그룹화 (Grouping)
# ------------------------------------------------------------------------------
print("\n'item_seq'를 기준으로 성분 정보와 제품명을 그룹화합니다...")

# 성분 정보를 딕셔너리로 만드는 함수 (이전과 동일)
def create_component_dict(row):
    return {
        'component_name': row['mtral_nm'],
        'quantity': row['qnt'],
        'unit': row['ingd_unit_cd']
    }

df['component_info'] = df.apply(create_component_dict, axis=1)

# 'item_seq'로 그룹화할 때, 'prduct' 정보와 'component_info' 정보를 함께 집계(aggregate)합니다.
# - prduct: 그룹 내 첫 번째 값을 대표로 사용 ('first')
# - component_info: 그룹 내 모든 값을 리스트로 묶음 (lambda x: list(x))
grouped_df = df.groupby('item_seq').agg(
    product_name=('prduct', 'first'),
    components_list=('component_info', lambda x: list(x))
).reset_index()

# 컬럼 이름 변경 (기존 코드와 통일성을 위해)
grouped_df.rename(columns={'product_name': 'item_name'}, inplace=True)

print("성분 정보 및 제품명 그룹화를 완료했습니다.")


# 4단계: 최종 저장
# ------------------------------------------------------------------------------
# 이제 'item_name' 컬럼이 포함된 최종 DataFrame이 생성됩니다.

# 효율적인 Parquet 포맷으로 저장 (이전과 동일)
try:
    save_filename = 'drug_data_processed_v4.parquet'
    grouped_df.to_parquet(save_filename, index=False)
    print(f"\n전처리 완료! 최종 데이터를 '{save_filename}' 파일로 저장했습니다.")
except ImportError:
    save_filename = 'drug_data_processed_v4.csv'
    grouped_df.to_csv(save_filename, index=False, encoding='utf-8-sig')
    print(f"\n[경고] pyarrow 라이브러리가 없어 CSV로 저장합니다.")
    print(f"전처리 완료! 최종 데이터를 '{save_filename}' 파일로 저장했습니다.")

# 결과 확인
print("\n--- 전처리 결과 샘플 (제품명 포함) ---")
print(grouped_df.head())

새로운 데이터('의약품 주성분 상세정보') 전처리를 시작합니다...
파일 로드 성공! 원본 데이터 형태: (129593, 13)

컬럼 이름을 소문자로 변경했습니다.
중복 제거 후 데이터 수: 124158개
'qnt' 컬럼을 정보 손실 없이 문자열 타입으로 변환했습니다.

'item_seq'를 기준으로 성분 정보와 제품명을 그룹화합니다...
성분 정보 및 제품명 그룹화를 완료했습니다.

전처리 완료! 최종 데이터를 'drug_data_processed_v4.parquet' 파일로 저장했습니다.

--- 전처리 결과 샘플 (제품명 포함) ---
    item_seq                                       item_name  \
0  195500005  중외5%포도당생리식염액(수출명:5%DextroseinnormalsalineInj.)   
1  195500006                                      중외5%포도당주사액   
2  195600004                                     중외20%포도당주사액   
3  195600006                                     중외50%포도당주사액   
4  195700004                                   대한포도당주사액(10%)   

                                     components_list  
0  [{'component_name': '포도당', 'quantity': '50', '...  
1  [{'component_name': '포도당', 'quantity': '50', '...  
2  [{'component_name': '포도당', 'quantity': '200', ...  
3  [{'component_name': '포도당', 'quantity': '500', ...  
4  [{'component_name': '포도당', 'qu

식품의약품안전처_의약품안전사용서비스(DUR)품목정보_병용금기 정보조회

In [ ]:
import requests
import json
import math
import time
import os
import ssl
from urllib.parse import unquote
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from google.colab import userdata

class CipherSuiteAdapter(HTTPAdapter):
    def __init__(self, *args, **kwargs):
        CIPHERS = ('ECDH+AESGCM:DH+AESGCM:ECDH+AES256:DH+AES256:ECDH+AES128:DH+AES:'
                   'ECDH+HIGH:DH+HIGH:RSA+AESGCM:RSA+AES:RSA+HIGH:!aNULL:!eNULL:!MD5:!3DES')
        self.ssl_context = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
        self.ssl_context.set_ciphers(CIPHERS)
        super().__init__(*args, **kwargs)

    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = requests.packages.urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize, block=block, ssl_context=self.ssl_context)

def download_all_drug_data_in_one_go(service_key, save_filename="usjnt_taboo_info_final.json"):
    """
    일일 트래픽(10,000회)이 충분하므로, 하루 만에 모든 데이터를 수집하는 최종 함수.
    (병용금기 DUR: getUsjntTabooInfoList03)
    """
    api_url_base = "https://apis.data.go.kr"
    # ★ 병용금기 엔드포인트
    api_path = "/1471000/DURPrdlstInfoService03/getUsjntTabooInfoList03"
    full_api_url = api_url_base + api_path
    decoded_service_key = unquote(service_key)

    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
    adapter = CipherSuiteAdapter(max_retries=retries)
    session.mount(api_url_base, adapter)

    all_items = []
    rows_per_page = 100

    # 1. 1페이지를 요청하여 전체 데이터 수를 정확히 파악
    print("데이터 수집을 시작합니다...")
    try:
        params = {'serviceKey': decoded_service_key, 'pageNo': '1', 'numOfRows': '1', 'type': 'json'}
        response = session.get(full_api_url, params=params, timeout=30)
        response.raise_for_status()
        data = response.json()

        # (최소 방어) response 래핑 대응
        header = data.get('header') or data.get('response', {}).get('header', {})
        body   = data.get('body')   or data.get('response', {}).get('body', {})

        if header.get('resultCode', '00') != '00':
            print(f"API 에러: {header.get('resultMsg')}")
            return False

        total_count = int(body.get('totalCount', 0))
        total_pages = math.ceil(total_count / rows_per_page)
        print(f"총 데이터: {total_count}개, 전체 페이지: {total_pages}개를 수집합니다.")
        print("-" * 50)
    except requests.exceptions.RequestException as e:
        print(f"초기 정보 파악 실패: {e}"); session.close(); return False
    except (ValueError, KeyError) as e:
        print(f"초기 응답 파싱 실패: {e}"); session.close(); return False

    # 2. 1페이지부터 마지막 페이지까지 순차적으로 수집
    for page in range(1, total_pages + 1):
        try:
            params = {'serviceKey': decoded_service_key, 'pageNo': str(page), 'numOfRows': str(rows_per_page), 'type': 'json'}
            print(f"페이지 {page}/{total_pages} 수집 중...")
            response = session.get(full_api_url, params=params, timeout=30)
            response.raise_for_status()
            data = response.json()

            header = data.get('header') or data.get('response', {}).get('header', {})
            body   = data.get('body')   or data.get('response', {}).get('body', {})

            items_data = body.get('items')
            # (보강) {"items":{"item":[...]}} 또는 dict 단건 형태도 처리
            if isinstance(items_data, dict) and 'item' in items_data:
                items_data = items_data['item']
            elif isinstance(items_data, dict):
                items_data = [items_data]

            if header.get('resultCode', '00') == '00' and items_data:
                if not isinstance(items_data, list): items_data = [items_data]
                all_items.extend(items_data)
            else:
                 print(f"  -> 페이지 {page}에서 데이터 없음: {header.get('resultMsg', '데이터 없음')}")

            # 서버 부하를 줄이기 위한 최소한의 딜레이
            time.sleep(0.2)

            # 100 페이지마다 긴 휴식을 주어 안정성 확보
            if page % 100 == 0 and page != total_pages:
                print(f"  -> {page} 페이지 도달. 서버 부하 방지를 위해 10초간 휴식합니다...")
                time.sleep(10)

        except (requests.exceptions.RequestException, json.JSONDecodeError) as e:
            print(f"  -> !!! 페이지 {page} 수집 실패: {e}. 이 페이지를 건너뜁니다.")
            continue

    # 3. 최종 저장
    session.close()
    try:
        print("-" * 50)
        print(f"총 {len(all_items)}개의 데이터를 성공적으로 수집했습니다.")
        print(f"'{save_filename}' 파일로 최종 저장합니다...")
        with open(save_filename, 'w', encoding='utf-8') as f:
            json.dump(all_items, f, ensure_ascii=False, indent=2)
        print(f"저장 완료! 모든 데이터 수집이 완료되었습니다!")
        return True
    except Exception as e:
        print(f"최종 파일 저장 중 에러 발생: {e}"); return False

# --- 스크립트 실행 부분 ---
if __name__ == "__main__":
    MY_SERVICE_KEY = userdata.get('MY_SERVICE_KEY')
    download_all_drug_data_in_one_go(service_key=MY_SERVICE_KEY)

In [ ]:
import pandas as pd
import json

print("병용금기(DUR) 데이터 전처리를 시작합니다...")

# --- 1단계: 파일 로드 ---
try:
    df = pd.read_json('usjnt_taboo_info_final.json', encoding='utf-8')
    print(f"파일 로드 성공! 원본 데이터 형태: {df.shape}")
except FileNotFoundError:
    print("'usjnt_taboo_info_final.json' 파일을 찾을 수 없습니다.")
    exit()

# --- 2단계: 기본 정제 ---
df.columns = [col.lower() for col in df.columns]
print("\n컬럼 이름을 소문자로 변경했습니다.")

before_dedup = len(df)
df.drop_duplicates(subset=['item_name', 'mixture_ingr_kor_name'], keep='first', inplace=True)
after_dedup = len(df)
print(f"중복 제거: {before_dedup}개 -> {after_dedup}개 ({before_dedup - after_dedup}개 제거)")

text_cols = ['prohbt_content', 'item_name', 'mixture_ingr_kor_name', 'item_seq']
for col in text_cols:
    if col in df.columns:
        df[col] = df[col].fillna('')
print("결측값 처리를 완료했습니다.")

# --- 3단계: 정보 구조화 ---
print("\n'item_name'을 기준으로 병용금기 정보를 그룹화합니다...")

def create_interaction_dict(row):
    return {
        'prohibited_ingredient': row['mixture_ingr_kor_name'],
        'reason': row['prohbt_content']
    }
df['interaction_info'] = df.apply(create_interaction_dict, axis=1)

grouped_df = df.groupby('item_name').agg(
    item_seq=('item_seq', 'first'),
    interactions_list=('interaction_info', list)
).reset_index()

print("정보 그룹화를 완료했습니다.")

# --- 4단계: 최종 저장 ---
final_df = grouped_df[['item_seq', 'item_name', 'interactions_list']]

# [핵심 수정] 나중에 통합할 때 이름이 충돌하지 않도록 미리 컬럼 이름을 변경합니다.
final_df.rename(columns={'interactions_list': 'interactions_list_2'}, inplace=True)
print("\n'interactions_list' 컬럼을 'interactions_list_2'로 변경했습니다.")

try:
    save_filename = 'drug_data_processed_v5.parquet'
    final_df.to_parquet(save_filename, index=False)
    print(f"\n전처리 완료! 최종 데이터를 '{save_filename}' 파일로 저장했습니다.")
except ImportError:
    save_filename = 'drug_data_processed_v5.csv'
    final_df.to_csv(save_filename, index=False, encoding='utf-8-sig')
    print(f"\n전처리 완료! 최종 데이터를 '{save_filename}' 파일로 저장했습니다.")

# --- 결과 확인 ---
print("\n--- 전처리 결과 샘플 ---")
print(final_df.head().to_string())

In [ ]:
import pandas as pd
import os
import ast
import numpy as np

print("의약품 데이터 통합 작업을 시작합니다. 🎉 (총 4개 파일, 상호작용 정보 분리 저장)")

# --- 1. 파일 경로 정의 ---
file_v2 = 'drug_data_processed_v2.parquet'
file_v3 = 'drug_data_processed_v3.parquet'
file_v4 = 'drug_data_processed_v4.parquet'
file_v5 = 'drug_data_processed_v5.parquet' # 이 파일은 'interactions_list_2' 컬럼을 가짐

# --- 2. 데이터 로드 및 Key 표준화 ---
def load_and_standardize(file_path):
    """
    Parquet 또는 CSV 파일을 로드하고, 'item_seq'를 문자열 형태로 표준화합니다.
    """
    csv_path = file_path.replace('.parquet', '.csv')
    try:
        df = None
        if os.path.exists(file_path):
            df = pd.read_parquet(file_path)
        elif os.path.exists(csv_path):
            df = pd.read_csv(csv_path, dtype={'item_seq': str})
        else:
            print(f"'{file_path}' 파일을 찾을 수 없습니다.")
            exit()

        if 'item_seq' in df.columns:
            df.dropna(subset=['item_seq'], inplace=True)
            df['item_seq'] = pd.to_numeric(df['item_seq'], errors='coerce').fillna(0).astype(np.int64).astype(str)

        return df
    except Exception as e:
        print(f"'{file_path}' 파일 로드 중 에러 발생: {e}")
        exit()

df_details = load_and_standardize(file_v2)
df_info = load_and_standardize(file_v3)
df_components = load_and_standardize(file_v4)
df_interaction = load_and_standardize(file_v5)

print("\n데이터 로드 완료! 데이터 병합을 시작합니다...")

# --- 3. 데이터 병합 (Merge) ---
# v2, v3, v4를 먼저 병합합니다.
merged_base = pd.merge(df_details, df_info, on='item_seq', how='outer', suffixes=('_v2', '_v3'))
merged_base = pd.merge(merged_base, df_components, on='item_seq', how='outer')
print(f"\n1차 병합(v2+v3+v4) 후: {merged_base.shape}")

# v5를 병합합니다. 'interactions_list'와 'interactions_list_2'는 이름이 달라 충돌 없이 각각의 컬럼으로 생성됩니다.
final_df = pd.merge(merged_base, df_interaction, on='item_seq', how='left')
print(f"병용금기 정보 병합 후: {final_df.shape}")


# --- 4. 중복 컬럼 정리 ---
print("\n중복된 컬럼들을 정리합니다...")

# item_name 및 나머지 중복/임시 컬럼 정리
final_df['item_name'] = final_df['item_name_v2'].fillna(final_df['item_name_v3']).fillna(final_df.get('item_name_x')).fillna(final_df.get('item_name_y'))
if 'item_name_clean_v2' in final_df.columns:
    final_df['item_name_clean'] = final_df['item_name_clean_v2'].fillna(final_df.get('item_name_clean_v3'))
if 'entp_name_v2' in final_df.columns:
    final_df['entp_name'] = final_df['entp_name_v2'].fillna(final_df.get('entp_name_v3'))

# 임시 컬럼 삭제
cols_to_drop = [col for col in final_df.columns if any(s in col for s in ['_v2', '_v3', '_x', '_y'])]
final_df.drop(columns=cols_to_drop, inplace=True, errors='ignore')

print("컬럼 정리 완료!")

# 최종 컬럼 순서 재배치 (두 개의 interactions_list를 모두 앞으로)
first_cols = ['item_seq', 'item_name', 'entp_name', 'interactions_list', 'interactions_list_2']
other_cols = [col for col in final_df.columns if col not in first_cols]
final_df = final_df[[col for col in first_cols if col in final_df.columns] + other_cols]

print(f"최종 데이터 형태: {final_df.shape}")
print("\n최종 컬럼 목록:", final_df.columns.tolist())

# --- 5. 최종 데이터 저장 ---
try:
    save_filename = 'drug_data_integrated_final.parquet'
    final_df.to_parquet(save_filename, index=False, compression='gzip')
    print(f"\n모든 데이터가 통합되었습니다! 최종 파일 '{save_filename}'(으)로 저장 완료!")
except:
    save_filename = 'drug_data_integrated_final.csv'
    final_df.to_csv(save_filename, index=False, encoding='utf-8-sig')
    print(f"\n모든 데이터가 통합되었습니다! 최종 파일 '{save_filename}'(으)로 저장 완료!")

# --- 결과 확인 ---
print("\n--- 통합 결과 샘플 (병용금기 정보 포함) ---")
# 두 리스트 중 하나라도 내용이 있는 경우를 샘플로 선택
sample_df = final_df[
    final_df['interactions_list'].notna() | final_df['interactions_list_2'].notna()
]

if not sample_df.empty:
    display_cols = ['item_name', 'interactions_list', 'interactions_list_2']
    pd.set_option('display.max_colwidth', 80)
    print(sample_df[display_cols].head(10).to_string())
else:
    print("병용금기 정보가 포함된 샘플을 찾지 못했습니다.")

In [ ]:
pip install pandas pyarrow sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 50.2 MB/s eta 0:00:00


챗봇

In [ ]:
import pandas as pd, re, html, numpy as np, ast

# ── 옵션 ──────────────────────────────────────
REMOVE_NUMBERS = True     # (1)(2)… 번호를 화면에서 제거
SORT_NUMBERED  = True     # 번호가 있으면 숫자순 정렬
TOP_K_RESULTS  = 3        # 약물명 검색 시 상위 매치 수
DATA_PATH      = "/content/drug_data_integrated_final.parquet"

# ── 데이터 로드 ───────────────────────────────
df = pd.read_parquet(DATA_PATH)

# ── 문자열/HTML 정리 ─────────────────────────
def s(x):
    """None/NaN/리스트/리스트 문자열 안전 문자열화"""
    if x is None or (isinstance(x, float) and np.isnan(x)): return ""
    if isinstance(x, list): return ", ".join(map(str, x))
    if isinstance(x, str):
        t = x.strip()
        if t in ("[]","[ ]","-",""): return ""
        if (t.startswith("[") and t.endswith("]")) or (t.startswith("(") and t.endswith(")")):
            try:
                val = ast.literal_eval(t)
                if isinstance(val, (list, tuple)): return ", ".join(map(str, val))
            except: pass
        return t
    return str(x)

def clean_html_keep_newlines(text: str) -> str:
    if not isinstance(text, str): return ""
    text = re.sub(r"<!\[CDATA\[(.*?)\]\]>", r"\1", text, flags=re.DOTALL)
    text = re.sub(r"</?(br|p|div|tr|table|ul|ol)>", "\n", text, flags=re.I)
    text = re.sub(r"<li[^>]*>", "\n- ", text, flags=re.I)
    text = re.sub(r"</li>", "\n", text, flags=re.I)
    text = re.sub(r"<[^>]+>", " ", text)
    text = html.unescape(text)
    text = text.replace("\r\n","\n").replace("\r","\n")
    text = re.sub(r"[ \t\u00A0]+"," ", text)
    text = re.sub(r"\n{3,}","\n\n", text)
    lines = [l.strip() for l in text.split("\n") if l.strip()]
    return "\n".join(lines)

# ── 불릿/번호 처리 ────────────────────────────
IDX = re.compile(r"^\s*(?:\(?\s*(\d{1,3})\s*\)|(\d{1,3})\.)\s*")
def extract_idx(t):
    m = IDX.match(t or "")
    if not m: return None, (t or "").strip()
    num = int(m.group(1) or m.group(2))
    return num, (t[m.end():]).strip()

def _bad_bullet(t:str)->bool:
    if not t: return True
    tt=t.strip().lower()
    return tt in {"[]","[ ]","-","·","•"} or tt.startswith("표") or tt in {"참고","주"}

def split_bullets(text:str):
    if not text: return []
    text=clean_html_keep_newlines(text)
    lines=[l.strip() for l in text.split("\n") if l.strip()]
    out=[]
    for ln in lines:
        # (?<=\))\s+(?=\d+\))|(?<=\.)\s+(?=\d+\.)|(?<=\))\s+(?=\(\d+\)) : (1) (2) 또는 1. 2. 또는 (1) (2) 패턴 분리
        # \s*[;•\u2022]\s+ : 세미콜론이나 불릿 기호로 분리
        # (?<=[.!?])\s+(?=[\(]?\d{1,3}[\)\.]\s) : 마침표/느낌표/물음표 뒤에 (1), 1. 같은 패턴이 오는 경우 분리
        parts=re.split(r"(?<=\))\s+(?=\d+\))|(?<=\.)\s+(?=\d+\.)|(?<=\))\s+(?=\(\d+\))", ln)
        for p in parts:
            subs=re.split(r"\s*[;•\u2022]\s+|(?<=[.!?])\s+(?=[\(]?\d{1,3}[\)\.]\s)", p)
            for sp in subs:
                sp=sp.strip(" •\u2022-")
                if len(sp)>1: out.append(sp)
    seen=set(); uniq=[]
    for t in out:
        _,body=extract_idx(t)
        key=body.lower()
        if not _bad_bullet(body) and key not in seen:
            uniq.append(t); seen.add(key)
    return uniq

def sort_numbered(lines):
    if not lines: return []
    enriched=[]; res=[]; block=[]; last=None
    for i,t in enumerate(lines):
        n,body=extract_idx(t)
        enriched.append((i,n,body,t))
    def flush():
        nonlocal res, block
        if not block: return
        if any(b[1] is not None for b in block):
            block=sorted(block,key=lambda x:(x[1],x[0]))
        res.extend([b[3] for b in block]); block=[]
        return []
    for rec in enriched:
        has = (rec[1] is not None)
        if last is None: last = has
        if has != last:
            flush(); last = has
        block.append(rec)
    flush()
    return res

def strip_numbers(lines):
    return [extract_idx(t)[1] for t in (lines or [])]

# ── 부작용 빈도 안내 문장 정리 ───────────────
_FREQ_PAT=re.compile(r"(발현\s*빈도|빈도)\s*[^.\n]*(≥?\s*1\s*/\s*10|1/10|1/1000|1/100|드물게|흔히)",re.I)
def normalize_frequency_legend(lines):
    cleaned=[]; legend=False
    for ln in lines:
        if _FREQ_PAT.search(ln):
            if not legend:
                cleaned.append("발현 빈도 안내: 매우 흔히(≥1/10), 흔히(≥1/100,<1/10), 드물게(<1/1,000).")
                legend=True
        else:
            cleaned.append(ln)
    return cleaned

# ── 부작용(증상) vs 금기/주의 판별 ───────────
ADVERSE_SYMPTOMS = [
    "발진","가려움","두드러기","구역","구토","복통","설사","식욕부진","어지럼",
    "피부","발열","쇽","쇼크","호흡곤란","간손상","간염","황달","혈소판 감소",
    "과립구감소","청색증","빈혈","췌장염","신장","간장애","위장출혈","천공",
    "스티븐스","리엘","메트헤모글로빈","과민반응","이상반응"
]
CONTRA_FLAGS = [
    "금기","투여 금지","복용 금지","주의","유의","환자 ", "환자는", "병력이 있는",
    "임부","수유부","고령자","소아","천식","신장장애","간장애","심장기능저하",
    "다음의 약물","복용한 환자","장기복용","과량","일일 최대 용량"
]
def is_contra_line(t: str) -> bool:
    tl = (t or "").lower()
    return any(k.lower() in tl for k in CONTRA_FLAGS)
def is_adverse_line(t: str) -> bool:
    tl = (t or "").lower()
    return any(k.lower() in tl for k in ADVERSE_SYMPTOMS) and not is_contra_line(t)

# ── 섹션 추출 (부작용·상호작용 통합) ───────
def extract_sections(row, max_items=12):
    eff_raw  = s(row.get("ee_doc_data",""))
    nb_raw   = s(row.get("nb_doc_data",""))
    inter1   = s(row.get("interactions_list",""))
    inter2   = s(row.get("interactions_list_2",""))

    # 효능
    eff = split_bullets(eff_raw)
    if SORT_NUMBERED: eff = sort_numbered(eff)
    if REMOVE_NUMBERS: eff = strip_numbers(eff)
    eff = [x for x in eff if x][:max_items]

    # 부작용(증상만; 금기/주의 문장은 제외)
    nb_lines = split_bullets(nb_raw)
    if SORT_NUMBERED: nb_lines = sort_numbered(nb_lines)
    if REMOVE_NUMBERS: nb_lines = strip_numbers(nb_lines)
    adverse = [x for x in nb_lines if is_adverse_line(x)]
    adverse = normalize_frequency_legend(adverse)

    # 상호작용(전용 컬럼만 사용)
    inter_src = " ".join([inter1, inter2]).strip()
    interaction = split_bullets(inter_src) if inter_src else []
    if SORT_NUMBERED: interaction = sort_numbered(interaction)
    if REMOVE_NUMBERS: interaction = strip_numbers(interaction)
    interaction = [x for x in interaction if x and "[]" not in x]

    # 부작용과 상호작용 통합
    combined_adv_inter_raw = adverse + interaction
    combined_adv_inter = []
    seen_lower = set() # 중복 제거를 위해 소문자 기준으로 비교
    for item in combined_adv_inter_raw:
        item_lower = item.lower()
        if item_lower not in seen_lower:
            combined_adv_inter.append(item)
            seen_lower.add(item_lower)
    combined_adv_inter = combined_adv_inter[:max_items]

    # (선택) 금기/주의: 보고 싶으면 반환값에 추가 사용 가능
    contraind = [x for x in nb_lines if is_contra_line(x)][:max_items]

    return eff, combined_adv_inter, contraind # 반환값을 변경: 효능, 통합 부작용/상호작용, 금기/주의

# ── 검색 준비 ─────────────────────────────────
SEARCH_COLS = [
    "item_name","item_name_clean","entp_name","ingr_name",
    "interactions_list","interactions_list_2",
    "ingredients_list","main_ingredients_list","components_list",
    "ee_doc_data","ud_doc_data","nb_doc_data"
]
def combine_text(row):
    parts=[]
    for col in SEARCH_COLS:
        if col in row and s(row[col]).strip():
            parts.append(s(row[col]).strip())
    return " ".join(parts)
df["search_text"]       = df.apply(combine_text, axis=1)
df["search_text_lower"] = df["search_text"].str.lower()

def search_drug(drug_name, top_k=TOP_K_RESULTS):
    q = drug_name.lower().strip()
    if not q: return None
    mask = df["search_text_lower"].str.contains(re.escape(q), na=False)
    res = df[mask].head(top_k)
    if not res.empty: return res
    mask2 = df["item_name"].astype(str).str.lower().str.contains(re.escape(q), na=False)
    res = df[mask2].head(top_k)
    return res if not res.empty else None

# ── 챗봇 루프 ─────────────────────────────────
print("\n의약품 정보 검색 챗봇을 시작합니다.")
print("종료하려면 '종료' 또는 'exit' 입력.\n")

while True:
    user_input = input("복용 중인 약물명 입력 (쉼표로 구분 가능): ").strip()
    if user_input.lower() in ["종료","exit"]:
        print("세션을 종료합니다."); break

    drugs = [x.strip() for x in re.split("[,;\n]+", user_input) if x.strip()]
    print(f"\n입력된 약물 목록: {drugs}")

    for d in drugs:
        results = search_drug(d)
        if results is None:
            print(f"\n'{d}' 관련 데이터가 없습니다."); continue

        print(f"\n'{d}' 관련 내용 (상위 {len(results)}개)\n{'-'*60}")
        for i, (_, row) in enumerate(results.iterrows(), 1):
            print(f"\n[{i}] 제품명: {s(row.get('item_name',''))} | 제조사: {s(row.get('entp_name',''))}")

            # extract_sections의 반환값이 변경되었으므로 할당도 변경해야 합니다.
            eff, combined_adv_inter, warn = extract_sections(row)

            print("\n효능:")
            print("\n".join(f"- {x}" for x in eff) if eff else "(정보 없음)")

            # 통합된 부작용 및 상호작용 출력
            print("\n부작용 및 상호작용(요약):")
            print("\n".join(f"- {x}" for x in combined_adv_inter) if combined_adv_inter else "(정보 없음)")

            # (선택) 금기/주의 — 필요 없으면 이 블록을 주석 처리해도 됨
            print("\n금기/주의(요약):")
            print("\n".join(f"- {x}" for x in warn) if warn else "(정보 없음)")

            print("-"*70)

    print("="*100)


의약품 정보 검색 챗봇을 시작합니다.
종료하려면 '종료' 또는 'exit' 입력.

복용 중인 약물명 입력 (쉼표로 구분 가능): 타이레놀

입력된 약물 목록: ['타이레놀']

'타이레놀' 관련 내용 (상위 3개)
------------------------------------------------------------

[1] 제품명: 티메롤8시간이알서방정(아세트아미노펜) | 제조사: (주)서울제약

효능:
- 해열 및 감기에 의한 동통(통증)과 두통, 치통, 근육통, 허리동통(통증), 생리통, 관절통의 완화

부작용 및 상호작용(요약):
- 매일 세잔 이상 정기적으로 술을 마시는 사람이 이 약이나 다른 해열 진통제를 복용해야 할 경우 반드시 의사 또는 약사와 상의해야 한다. 이러한 사람이 이 약을 복용하면 간손상이 유발될 수 있다.
- 혈액: 혈소판 감소, 과립구감소, 용혈성빈혈, 메트헤모글로빈혈증, 혈소판기능 저하(출혈시간 연장), 청색증
- 과민증: 과민증상(얼굴부기, 호흡곤란, 땀이 남, 저혈압, 쇽)
- 피부: 발진, 알레르기 반응, 피부점막안 증후군(스티븐스-존슨 증후군), 중독성표피괴사증(리엘 증후군)
- 기타: 장기투여시 만성간괴사, 급성췌장(이자)염, 만성간염, 신장독성
- 발현 빈도 안내: 매우 흔히(≥1/10), 흔히(≥1/100,<1/10), 드물게(<1/1,000).
- 피부 및 피하조직 장애
- 매우 드물게 : 두드러기, 소양성(가려움) 발진, 발진
- 국내 부작용 보고자료의 분석·평가에 따라 다음의 이상반응을 추가한다.
- ⦁ 피부: 고정발진
- 가. 급성통증 및 발열의 정도를 고려하여 복용한다.
- 의사 또는 약사의 지시없이 통증에 10일 이상(성인) 복용하지 않고 발열에 3일 이상 복용하지 않는다. 통증이나 발열 증상이 지속되거나 악화될 경우, 또는 새로운 증상이 나타날 경우 의사 또는 약사와 상의한다.

금기/주의(요약):
- 아세트아미노펜을 복용한 환자에서 매우 드물게 급성 전신성 발진성 농포증(급성 전신성 발진성 고름